['ACRES HOMES ID', 'ADDICKS UTILITY DISTRICT']

In [6]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

empty_dww = pd.DataFrame()
empty_district_info = pd.DataFrame()
empty_audit_info = pd.DataFrame()
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
base_page = 'http://www14.tceq.texas.gov/iwud/dist/index.cfm?fuseaction=ListDistricts&COMMAND=LIST&compress=N&StartName=&ID=&RegionCode=&DistTypeCode=&CreationTypeCode=&DistFunctionTypeCode=&CountyCode=&FinancialStatus=&ActivityStatus=&ListStart='
driver.get(base_page)
#CountyCode

houston_counties = [79,101,170]

from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
#
# select by value 
#select.select_by_value('1')
#wait = WebDriverWait(driver, 10)
select = Select(driver.find_element_by_css_selector('#CountyCode'))
wd_info_call = '.iwuddata:nth-child(2)'
#wd_info_header = 'tr:nth-child(5) .iwudheading , .iwudheading:nth-child(1)'
wd_info_header = '.iwudheading:nth-child(1)'
#for county in houston_counties:
count = 0
select_county = Select(driver.find_element_by_css_selector('#CountyCode'))
select_county.select_by_value(str(101))
driver.find_elements_by_css_selector('input')[1].click()
css_find_wds = "table+ table .iwud a"
css_find_number = '.iwud+ .iwud:nth-child(3)'
css_find_status = '.iwud~ .iwud+ .iwud'
css_to_click_more = "table+ table tr:nth-child(1) a:nth-child(1) img"
audit_vars_css = 'td.iwud:nth-child(1)'
audit_vals_css = '.iwud+ td'
get_doc_id = 'table:nth-child(6) tr:nth-child(1) .iwuddata'

System = [];Status = []; ID = []
#### Grab basic district info
count = 0

for system in range(2000):
    print('system' + str(system))
    temp_wd = driver.find_elements_by_css_selector(css_find_wds)
    temp_status = driver.find_elements_by_css_selector(css_find_status)
    temp_number = driver.find_elements_by_css_selector(css_find_number)
    System.append(temp_wd[count].text)
    Status.append(temp_status[count].text)
    ID.append(temp_number[count].text)
    water_systems = driver.find_elements_by_css_selector(css_find_wds)
    #for systems in range(len(water_systems)):
    temp_name = driver.find_elements_by_css_selector(css_find_wds)[count].text
    print(temp_name)
    temp_id = driver.find_elements_by_css_selector(css_find_number)[count].text
    driver.find_elements_by_css_selector(css_find_wds)[count].click()
    info_vars = []
    info = []
    for el in driver.find_elements_by_css_selector(wd_info_call):
        info.append(el.text)
    for el in driver.find_elements_by_css_selector(wd_info_header):
        info_vars.append(el.text)
    i = 0
    for v in range(len(info_vars)):
        if info_vars[v] == '':
            i = i + 1
            info_vars[v] = str(i)
            #print(info_vars);print(info)
    temp = pd.DataFrame(info).T
    temp.columns = [info_vars]
    temp['NAME'] = temp_name
    temp['WDD_ID'] = temp_id
    #for dup in range(sum(temp.columns.duplicated())):
    while sum(temp.columns.duplicated())>0:
        temp = temp.drop(temp.columns[temp.columns.duplicated()][0],axis=1)
        #temp = temp.drop('Address:', axis=1)
        #temp = temp.columns.drop_duplicates()
        #print(temp_info)
        #if empty_dww.shape[0] !=0 & any(empty_dww['Name'] == temp_name):
        #    driver.back()
        #    continue  join(temp_info,how='outer')
        #print(temp.columns)
    empty_district_info = pd.concat([empty_district_info,temp], axis=0, join='outer', ignore_index=True)
    empty_district_info.to_csv('../Input/tceq_audits/harris_system_info.csv')
        #empty_district_info = empty_district_info.append(temp,how='outer')
        #empty_district_info =  = empty_district_info.append(temp)
        #grouped = empty_district_info.groupby(level=0)
        #empty_district_info = grouped.last()
    driver.find_element_by_css_selector('.iwud a+ a img').click()
        #doc_list = driver.find_elements_by_css_selector('.iwudcolor+ tr td')
    audit_links = driver.find_elements_by_partial_link_text('AUDIT')
    if (len(audit_links)==0):
        driver.back()
    if (len(audit_links)!=0):
        for audit in range(len(audit_links)):
            audit_links = driver.find_elements_by_partial_link_text('AUDIT')
            doc_link = audit_links[audit].get_attribute('href')
            audit_links[audit].click()   
            avars = []; avals = []
            doc_id = driver.find_element_by_css_selector(get_doc_id).text
            #print('docid')
            #print(doc_id)
            for var in driver.find_elements_by_css_selector(audit_vars_css):
                avars.append(var.text)
            for var in driver.find_elements_by_css_selector(audit_vals_css):
                avals.append(var.text)
            temp_audit = pd.DataFrame(avals).T
            temp_audit.columns = avars
            temp_audit['DOC_ID'] = doc_id
            temp_audit['SYSTEM_NAME'] = temp_name
            temp_audit['SYSTEM_ID'] = temp_id
            temp_audit['DOC_LINK'] = doc_link
                #temp_audit,index=avars,columns = [doc_id])
                #temp_audit = temp_audit.append(pd.DataFrame([id],index=['System ID'],columns=[doc_id]))
                #temp_audit = temp_audit.append(pd.DataFrame([doc_link],index=['Doc_Link'],columns=[doc_id]))
            empty_audit_info = pd.concat([empty_audit_info,temp_audit],axis=0, join='outer', ignore_index=True)
            empty_audit_info.to_csv('../Input/tceq_audits/harris_audits.csv')
            driver.back()
        driver.back()
    driver.back()
    if len(temp_wd)<49 & count == len(temp_wd):
        break
    if count == 49:
        driver.find_element_by_css_selector(css_to_click_more).click()
        count = 0
        continue
    count = count + 1
system_list_df = pd.DataFrame({'Name': System, 'Status': Status,'ID': ID,'County': 'HARRIS'})
#empty_dww = empty_dww.append(system_list_df)       
#harris_dww = system_list_df
system_list_df.to_csv('../Input/tceq_audits/harris_base.csv')
#harris_audits = empty_audit_info
#harris_systems = empty_district_info


#while len(driver.find_element_by_css_selector(css_to_click_more))>0:
#    driver.find_element_by_css_selector(css_to_click_more).click()



system0
ACRES HOMES ID
system1
ADDICKS UTILITY DISTRICT
system2
AIRLINE IMPROVEMENT DISTRICT
system3
ALDINE MUD
system4
ALDINE MUD 1
system5
ALDINE PUD
system6
ARCADIAN GARDENS MUD
system7
BAKER ROAD MUD
system8
BAMMEL UTILITY DISTRICT
system9
BAMMEL UTILITY DISTRICT
system10
BARKER CYPRESS MUD
system11
BARRETT MANAGEMENT DISTRICT
system12
BAUER LANDING WCID
system13
BAYBROOK MANAGEMENT DISTRICT
system14
BAYBROOK MUD 1
system15
BAYFIELD PUD
system16
BAYSHORE MUD
system17
BAYTOWN AREA WATER AUTHORITY
system18
BEAR CREEK MUD
system19
BEAUMONT PLACE UTILITY DISTRICT
system20
BEECHNUT MUD
system21
BELTWAY 8 MUD 1
system22
BELTWAY 8 MUD 2
system23
BELTWAY 8 MUD 3
system24
BELTWAY MUD
system25
BILMA PUD
system26
BISSONNET MUD
system27
BOONE ROAD UTILITY DISTRICT
system28
BRAES UTILITY DISTRICT
system29
BRAZORIA COUNTY MUD 23
system30
BRIAR RIDGE MUD
system31
BRIARWICK IMPROVEMENT DISTRICT
system32
BRIDGELAND MANAGEMENT DISTRICT
system33
BRIDGESTONE MUD
system34
BROOKFIELD MUD
system35
BUFFAL

IndexError: list index out of range

In [7]:
system_list_df = pd.DataFrame({'Name': System, 'Status': Status,'ID': ID,'County': 'HARRIS'})
#empty_dww = empty_dww.append(system_list_df)       
#harris_dww = system_list_df
system_list_df.to_csv('../Input/tceq_audits/harris_base.csv')